## Installing dependencies

In [ ]:
#downloading libraries
import pandas as pd 
import numpy as np
import matplotlib as plt
import pandas as pd
import numpy as np

## First approach regarding data extraction -

* Basically parsed through top,hot and new of r/india on reddit and appended data elements of a post if it hadnt already occured using the PRAW reddit API



In [ ]:
client_id = 'F9GtKXXXXXXXXw'
secret = 'hRtKBcXXXXXXXXXXZFX8OHCpkxY'
name = 'jay.0tist'

reddit = praw.Reddit(user_agent='funky_cool_bruh', client_id=client_id, client_secret=secret,
                     username='jayesh0vasudeva', password='jaiXXXXXXXXJAI')

In [ ]:
import praw
import pprint
import pandas as pd

posts = []
india_subreddit = reddit.subreddit('india')
for post in india_subreddit.top(limit=1000):
    posts.append([post.title, post.score, post.id, post.url, post.num_comments, post.selftext, post.created,post.link_flair_text])
    
posts_pd = pd.DataFrame(posts,columns=['title', 'score', 'id', 'url', 'num_comments', 'body', 'created','flair'])

for post in india_subreddit.hot(limit=1000):
    if post.title not in posts_pd['title'].unique():
         posts.append([post.title, post.score, post.id, post.url, post.num_comments, post.selftext, post.created,post.link_flair_text])
            
posts_pd = pd.DataFrame(posts,columns=['title', 'score', 'id', 'url', 'num_comments', 'body', 'created','flair'])

for post in india_subreddit.new(limit=1000):
    if post.title not in posts_pd['title'].unique():
         posts.append([post.title, post.score, post.id, post.url, post.num_comments, post.selftext, post.created,post.link_flair_text])
            
posts_pd = pd.DataFrame(posts,columns=['title', 'score', 'id', 'url', 'num_comments', 'body', 'created','flair'])

* One problem that occured while doing this was that there were some random flairs , so in order to overcome that i either converted them into a known flair that was similar enough or I completely removed them 

* I was able to collect 2854 different posts using this method , but there was a very uneven divide in the flairs as some of them had over 800 instances while some had as few as 20


In [ ]:
i=0
for flair in posts_pd['flair']:
    if flair == 'Unverified' or flair == 'AMA' or flair == 'AMA Concluded':
        posts_pd['flair'][i]='AMA'
        print(posts_pd['flair'][i])
    i=i+1
i=0
for flair in posts_pd['flair']:
    if flair == 'Policy & Economy' or flair == 'Policy' or flair == 'Demonetization' or flair == 'Policy/Economy ':
        posts_pd['flair'][i]='Policy/Economy'
        print(posts_pd['flair'][i])
    i=i+1   
i=0
for flair in posts_pd['flair']:
    if flair == 'Entertainment' or flair == 'Misleading' or flair == 'r/all' or flair == '/r/all':
        posts_pd['flair'][i]='Non-Political'
        print(posts_pd['flair'][i])
    i=i+1    
i=0
for flair in posts_pd['flair']:
    if flair == 'Science & Technology':
        posts_pd['flair'][i]='Science/Technology'
        print(posts_pd['flair'][i])
    i=i+1    
    
    
flair_list=['AMA','AskIndia','Business/Finance','Food','Non-Political','Photography','Policy/Economy','Politics','Science/Technology','Sports','[R]eddiquette']
i=0
for flair in posts_pd['flair']:
    if flair not in flair_list:
        posts_pd['flair'][i]='waste'
        print(posts_pd['flair'][i])
    i=i+1  

temp=[]
for i in range(len(posts_pd)):
    if posts_pd['flair'][i]!='waste':
        temp.append([posts_pd.title[i], posts_pd.score[i], posts_pd.id[i], posts_pd.url[i], posts_pd.num_comments[i], posts_pd.body[i], posts_pd.created[i],posts_pd.flair[i]])
posts_corr= pd.DataFrame(temp,columns=['title', 'score', 'id', 'url', 'num_comments', 'body', 'created','flair'])

In [ ]:
## The second method of extracting text -
 
* This is the approach that worked for me 

* Here using the PRAW API I parse through reddit posts with specific flairs and collect 100 data points per flair with various data elements

In [ ]:
reddit = reddit = praw.Reddit(client_id='300HIOocldVcKA', client_secret='PCktLUhpPaRB1RrBCouEDPdpEBU', user_agent='abhishek chopra')
flair_list=['AMA','AskIndia','Business/Finance','Food','Non-Political','Photography','Policy/Economy','Politics','Science/Technology','Sports','[R]eddiquette']
subreddit = reddit.subreddit('india')
topics_dict = {"flair":[], "title":[], "score":[], "id":[], "url":[], "body":[],"comms_num": [], "created": [], "comments":[]}
import datetime as dt
def date_convertor(data):
    return dt.datetime.fromtimestamp(data)

for flair in flair_list:
  
  get_subreddits = subreddit.search(flair, limit=100)
  
  for submission in get_subreddits:
    
    topics_dict["flair"].append(flair)
    topics_dict["title"].append(submission.title)
    topics_dict["score"].append(submission.score)
    topics_dict["id"].append(submission.id)
    topics_dict["url"].append(submission.url)
    topics_dict["body"].append(submission.selftext)
    topics_dict["comms_num"].append(submission.num_comments)
    topics_dict["created"].append(submission.created)
    
    submission.comments.replace_more(limit=None)
    comment = ''
    for top_level_comment in submission.comments:
        comment = comment + ' ' + top_level_comment.body
    topics_dict["comments"].append(comment)
    
topics_data = pd.DataFrame(topics_dict)
_timestamp = topics_data["created"].apply(date_convertor)
topics_data = topics_data.assign(timestamp = _timestamp)
del topics_data['created']
topics_data.to_csv('reddit-india-data.csv', index=False)